In [13]:
import pandas as pd
import os
from sqlalchemy import create_engine, inspect
from sqlalchemy.engine import URL
import logging
import time

# ---------- Logging ----------
os.makedirs("logs", exist_ok=True)

logging.basicConfig(
    filename="logs/project_business.log",
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    filemode="a"
)

# ---------- MySQL Connection ----------
url = URL.create(
    drivername="mysql+pymysql",
    username="root",
    password="Macebox@1315",
    host="localhost",
    port=3306,
    database="project_business"
)

engine = create_engine(
    url,
    pool_recycle=3600,
    pool_pre_ping=True,
    echo=False,
    future=True
)

# ---------- Connection Test ----------
try:
    with engine.connect():
        print("✅ MySQL Connected Successfully!")
except Exception as e:
    print("❌ Connection Error:", e)
    raise SystemExit(e)


# ---------- Upload Function ----------
def ingest_excel(file_path, table_name, engine):

    inspector = inspect(engine)

    if table_name in inspector.get_table_names():
        print(f"⚠️ Table '{table_name}' already exists — skipping...")
        logging.warning(f"Table {table_name} already exists.")
        return

    try:
        start_time = time.time()

        # ⭐ Read Excel
        df = pd.read_excel(file_path)

        print(f"📊 Uploading {table_name} | Rows: {len(df)}")
        logging.info(f"Uploading {table_name} with {len(df)} rows")

        # ⭐ SAFE chunk size
        chunk_size = 20000

        for start in range(0, len(df), chunk_size):
            chunk = df.iloc[start:start+chunk_size]

            chunk.to_sql(
                table_name,
                con=engine,
                if_exists='append',
                index=False,
                method='multi'
            )

        total = (time.time() - start_time) / 60

        print(f"✅ {table_name} uploaded in {total:.2f} minutes")
        logging.info(f"{table_name} uploaded successfully in {total:.2f} minutes")

    except Exception as e:
        print(f"❌ Error uploading {table_name}: {e}")
        logging.error(f"Error uploading {table_name}: {e}")


# ---------- Load All Excel Files ----------
def load_raw_data():

    data_folder = "data"
    start = time.time()

    if not os.path.exists(data_folder):
        raise FileNotFoundError("❌ 'data' folder not found!")

    for file in os.listdir(data_folder):

        if file.endswith(".xlsx"):   # ✅ CORRECT EXTENSION

            file_path = os.path.join(data_folder, file)

            table_name = file.replace(".xlsx", "").lower()

            logging.info(f"🚀 Uploading {table_name} ...")
            ingest_excel(file_path, table_name, engine)

    total_time = (time.time() - start) / 60

    logging.info("🔥 ALL DATA UPLOADED SUCCESSFULLY!")
    logging.info(f"Total Time Taken {total_time:.2f} minutes")

    print(f"\n🔥 ALL DATA UPLOADED IN {total_time:.2f} MINUTES")


# ---------- Run ----------
load_raw_data()

✅ MySQL Connected Successfully!
📊 Uploading customers_table | Rows: 3005
✅ customers_table uploaded in 0.05 minutes
📊 Uploading orders_table | Rows: 20118
✅ orders_table uploaded in 0.15 minutes
📊 Uploading products_table | Rows: 199
✅ products_table uploaded in 0.00 minutes
📊 Uploading region_table | Rows: 8
✅ region_table uploaded in 0.00 minutes
📊 Uploading returns_table | Rows: 20
✅ returns_table uploaded in 0.00 minutes

🔥 ALL DATA UPLOADED IN 0.21 MINUTES
